In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是由[Mohammad Al-Ansari](https://github.com/Mansari)贡献的版本升级。特别感谢[Andrew Ferlitsch](https://github.com/andrewferlitsch)对他的审查和编辑。

这是[Vertex AI SDK for Python：用于在线预测笔记本的AutoML训练文本实体抽取模型](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_text_entity_extraction_online.ipynb)的扩展，最初由[Andrew Ferlitsch](https://github.com/andrewferlitsch)和[Karl Weinmeister](https://github.com/kweinmeister)合著。这个版本增加了使用`Vision API`和`BigQuery`来预处理`Vertex AI AutoML`数据集，以用于文本实体抽取模型的训练。

# GCP上的E2E ML：MLOps阶段2：实验：开始使用Vision API测试预处理和AutoML文本模型生成

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_visionapi_and_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_visionapi_and_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_visionapi_and_automl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用`BigQuery`，`Vision AI`和`Vertex AI SDK` for Python来基于现有训练数据训练一个文本实体提取模型。

### 目标

在本教程中，您将通过生成自定义导入文件来创建一个`AutoML`文本实体提取模型，该模型使用预先提取的数据。您将使用`BigQuery`、`Vision AI`、Cloud Storage和Python的`Vertex AI SDK`来从Python脚本中部署此模型以进行在线预测。您也可以选择使用`gcloud`命令行工具或在Cloud Console中在线创建和部署模型。

使用先前进行过标注的现有训练数据在训练模型时非常有用，因为它允许您使用更大的数据集而资源需求较小。

本教程使用以下Google Cloud服务：

- `BigQuery`
- `Vision AI`
- `Vertex AI AutoML`

所执行的步骤包括：

- 使用`Vision AI` API对训练文件进行预处理，从PDF文件中提取文本。
- 创建一个自定义导入文件，其中包含基于示例`BigQuery`数据集的注释数据。
- 创建一个`Vertex AI Dataset`资源。
- 训练模型。
- 查看模型评估。
- 部署`Vertex AI Model`资源到一个提供`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 数据集

本教程使用的数据集是来自Google公共数据集的[带有提取结构化数据的专利PDF样本](https://console.cloud.google.com/marketplace/product/global-patents/labeled-patents)。

这个数据集包括从美国和欧盟发行的300多份专利文件中提取的数据。数据集包括每份专利的第一页的Cloud Storage存储区块的链接，以及一些提取的实体。

这些数据被发布为[公共数据集](https://cloud.google.com/bigquery/public-data) 在`BigQuery`上。

成本

本教程使用Google Cloud的可计费组件：

* BigQuery
* Vision API
* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Vision API 价格](https://cloud.google.com/vision/pricing)，[BigQuery 价格](https://cloud.google.com/bigquery/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已经满足运行此笔记本的所有要求。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- BigQuery SDK
- Vision API SDK
- Vertex AI SDK
- Cloud Storage SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Cloud Storage指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简要的指导：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装执行此笔记本所需的软件包。您可以忽略 pip 依赖解析器的错误，因为它们不会影响此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery google-cloud-vision google-cloud-storage pandas $USER_FLAG -q

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果您有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您在哪个笔记本环境中，下面的步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得$300的免费信用用于您的计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：BigQuery API、Vision API、Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,vision.googleapis.com,aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter会将以`!`开头的行视为shell命令，并会插入以`$`开头的Python变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

#### 视觉人工智能

您现在可以通过设置`VISION_AI_REGION`变量来指定洲级数据存储和光学字符识别（OCR）处理。您可以选择以下选项之一：

* 仅限美国：`us`
* 欧洲联盟：`eu`

了解更多关于[OCR的Vision AI区域](https://cloud.google.com/vision/docs/pdf#regionalization)

#### 顶点人工智能

您还可以更改`VERTEX_AI_REGION`变量，该变量用于本笔记本之后的操作。以下是支持Vertex AI的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex AI的训练中使用多区域存储桶。并非所有区域都支持所有的Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
VISION_AI_REGION = "[your-region]"  # @param {type: "string"}

if VISION_AI_REGION == "[your-region]":
    VISION_AI_REGION = "us"

VERTEX_AI_REGION = "[your-region]"  # @param {type: "string"}

if VERTEX_AI_REGION == "[your-region]":
    VERTEX_AI_REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench**，您的环境已经通过验证。请跳过这一步。如果您仍然收到错误消息，您可能需要授予运行工作台笔记本的服务帐户访问下面列出的服务。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，然后选择**Vertex AI管理员**。在筛选框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源所关联的所有数据在会话间保留的地方。此桶还将用于存储 Vision API SDK 将 PDF 转换为文本的输出。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶尚不存在时才能执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $VERTEX_AI_REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import os

from google.cloud import aiplatform, bigquery, storage, vision

初始化Python的BigQuery SDK

创建BigQuery客户端。

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

### 初始化Python的Vision API SDK

In [ ]:
vision_client_options = {
    "quota_project_id": PROJECT_ID,
    "api_endpoint": f"{VISION_AI_REGION}-vision.googleapis.com",
}
vision_client = vision.ImageAnnotatorClient(client_options=vision_client_options)

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(
    project=PROJECT_ID, location=VERTEX_AI_REGION, staging_bucket=BUCKET_URI
)

### 初始化Python云存储SDK

In [ ]:
storage_client = storage.Client(project=PROJECT_ID)

## 教程

现在你已经准备好开始用自定义导入文件创建自己的AutoML文本实体提取模型。

查询专利PDF样本数据集

首先，您可以运行查询以选择数据的子集以用于训练目的。

In [ ]:
# Select only patents issued in the US and in English
query = """
    SELECT * FROM `bigquery-public-data.labeled_patents.extracted_data`
    WHERE issuer = 'US' and language = 'EN'
    ORDER BY gcs_path ASC
    """

query_job = bq_client.query(query)

# Convert the results into a pandas dataframe
results_df = query_job.result().to_dataframe()

print(f"Retrieved {len(results_df)} rows")

#### 查看数据

您可以通过展示前5行数据来快速查看数据的内容。

In [ ]:
print(results_df.head(5))

`gcs_path`包含了PDF文件的完整路径。为了让您的生活更轻松，您将向数据框添加另一列，其中仅包含文件名（无扩展名）。您可以使用pandas进行此操作。

了解有关[pandas文本处理](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html)的更多信息。

In [ ]:
# Apply a transformation and store the result in a new column
results_df["gcs_filename"] = results_df["gcs_path"].apply(
    lambda x: os.path.splitext(os.path.basename(x))[0]
)

# Take a look at the data to see the newly added filename at the end
print(results_df.head(5))

### 创建自动机器学习数据集导入文件

现在我们已经准备好数据，我们可以继续下一步。

#### 使用Vision API对文件进行预处理

Vertex AI需要纯文本来训练文本实体提取模型。然而，我们的样本数据集并不包含用于提取数据的完整文本，而是包含PDF版本的链接。

[这是一个PDF文件的示例](https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/community/ml_ops/stage2/notebook_resources/get_started_with_visionapi_and_automl/us_001.pdf)

您需要将PDF文件转换为文本，以便在构建注释文件时使用它。您可以使用`Vision AI`文档文本检测来实现此目的。

了解更多关于[检测文件中的文本（PDF/TIFF）](https://cloud.google.com/vision/docs/pdf)。

In [ ]:
# Specify a destination path inside our bucket
gcs_destination_path = "ocr-output"
gcs_destination_uri = f"{BUCKET_URI}/{gcs_destination_path}"

# Specify the feature for the Vision API processor
feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

# Create a collection of requests. The SDK requires a separate request per each
# file that we want to extract text from
async_requests = []

# Build the requests array by iterating through our dataset
for i in range(len(results_df)):
    gcs_uri = results_df.loc[i, "gcs_path"]

    # Build input_config
    gcs_source = vision.GcsSource(uri=gcs_uri)
    input_config = vision.InputConfig(
        gcs_source=gcs_source, mime_type="application/pdf"
    )

    # Build output config
    gcs_source_filename = results_df.loc[i, "gcs_filename"]
    gcs_destination = vision.GcsDestination(
        uri=f"{gcs_destination_uri}/{gcs_source_filename}-"
    )
    output_config = vision.OutputConfig(gcs_destination=gcs_destination)

    # Build request object and add to the collection
    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config, output_config=output_config
    )

    async_requests.append(async_request)

print(f"Created {len(async_requests)} requests")

# Submit the batch OCR job
operation = vision_client.async_batch_annotate_files(requests=async_requests)
print("Submitting the batch OCR job")

print("Waiting for the operation to finish... this will take a short while")

response = operation.result(timeout=420)

print("Completed!")

### 生成导入文件

由于您已经从专利中提取了数据，您可以使用这些数据生成注释，以用于训练`AutoML`模型。

您可以通过查找转换文本中的特定提取实体，然后指定文档中的文本位置来执行此操作。

#### 生成注释

在本教程中，您只需注释三个实体：发表日期、申请号和发明人姓名的第一行。您可以扩展代码以注释提取数据集中可用的其他实体。

In [ ]:
import json

# Remove the leading gs:// from the bucket name by splitting the name
# and taking the second part
gcs_bucket_name = BUCKET_URI.split("gs://")[1]

output_bucket = storage_client.bucket(gcs_bucket_name)
# print(output_bucket)
print("Creating annotation set based on output text files and dataset")

annotations = []

# Loop through the dataset
for i in range(len(results_df)):
    gcs_uri = results_df.loc[i, "gcs_path"]

    # For each row, try to read the resulting text file from the output directory
    # based on some path name
    gcs_output_filename = f"{results_df.loc[i, 'gcs_filename']}-output-1-to-1.json"
    blob_name = f"{gcs_destination_path}/{gcs_output_filename}"
    blob = output_bucket.blob(blob_name)
    contents = blob.download_as_string()

    json_object = json.loads(contents)

    full_text = json_object["responses"][0]["fullTextAnnotation"]["text"]

    text_segment_annotations = []

    # Prepare labels to be annotated
    labels = [
        {
            "name": "lbl_publication_date",
            "value": results_df.loc[i, "publication_date"],
        },
        {
            "name": "lbl_application_number",
            "value": results_df.loc[i, "application_number"],
        },
        {"name": "lbl_inventor_line_1", "value": results_df.loc[i, "inventor_line_1"]},
    ]

    # Search for the labels in the patent text
    for label in labels:
        startIndex = 0

        while True:
            found_index = full_text.find(label["value"], startIndex)
            if found_index == -1:
                break
            else:
                end_offset = found_index + len(label["value"])
                text_segment_annotations.append(
                    {
                        "startOffset": found_index,
                        "endOffset": end_offset,
                        "displayName": label["name"],
                    }
                )
                # Move the startIndex in the search to the end offset so we can
                # find the next match
                startIndex = end_offset

    # Create the annotation object and add to the collection
    annotation = {
        "textSegmentAnnotations": text_segment_annotations,
        "textContent": full_text,
    }

    annotations.append(annotation)

print("Done !")

生成并上传导入文件

现在您已经有了注释，您可以根据[此指南](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#entity-extraction)中提到的结构生成导入文件。

您可以将您的导入文件上传到您的存储桶，以用于`AutoML`训练。

In [ ]:
# Convert array to JSONL content
jsonl_output = ""
for annotation in annotations:
    jsonl_output += json.dumps(annotation) + "\n"

print(f"Created import file based on {len(annotations)} annotations")

# Upload content to GCS to be used in our next step
gcs_annotation_file_name = "annotation_file/import_file.jsonl"
import_file_blob = output_bucket.blob(gcs_annotation_file_name)
import_file_blob.upload_from_string(jsonl_output)

print(f"Uploaded import file to {output_bucket.name}/{gcs_annotation_file_name}")

### 创建Vertex AI数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，需要提供以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要十至二十分钟。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="Patent PDF Samples" + "_" + TIMESTAMP,
    gcs_source=[f"gs://{output_bucket.name}/{gcs_annotation_file_name}"],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.extraction,
)

print(dataset.resource_name)

现在数据集已经导入，您还可以在云控制台中查看数据集的注释，以验证专利文本已成功注释。

### 创建和运行`AutoML`训练流水线

为了训练一个`AutoML`模型，您需要执行两个步骤：1) 创建一个`AutoML`训练流水线，和2) 运行这个流水线。

#### 创建训练流水线

使用`AutoMLTextTrainingJob`类创建一个`AutoML`训练流水线，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，是单标签(False)还是多标签(True)。
- `sentiment_max`：如果是情感分析任务，情感值的最大值。

In [ ]:
job = aiplatform.AutoMLTextTrainingJob(
    display_name="patent_sample_" + TIMESTAMP, prediction_type="extraction"
)

print("Training job created!")

#### 运行训练流程

接下来，您可以通过调用`run`方法来开始训练作业，使用以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的人类可读名称。
- `training_fraction_split`: 用于训练的数据集所占的百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集所占的百分比。
- `validation_fraction_split`: 用于验证的数据集所占的百分比。

当`run`方法完成时，将返回`Model`资源。

训练流程的执行时间最长为4小时。

_如果使用Colab，请确保您的会话在等待模型训练完成时不会超时。您可能会遇到问题，需要重新初始化笔记本，并手动设置下一步中的模型评估。_

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="patent_sample_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 检查模型评估分数

在您的模型训练完成后，您可以使用`list_model_evaluations()`方法来检查其评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

将`AutoML`模型部署到`Vertex AI Endpoint`资源

接下来，将您的`AutoML`模型部署到`Vertex AI Endpoint`资源以进行在线预测。要部署模型，您需要调用`deploy`方法。

这将需要几分钟时间。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

发送一个在线预测请求到你部署的模型。

### 创建一个测试项目

接下来，您将使用从专利的第一页提取的文本运行一个测试，该专利不是我们原始数据集的组成部分。 [这是一个将PDF文件转换为文本的链接。](https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/community/ml_ops/stage2/notebook_resources/get_started_with_visionapi_and_automl/sample_patent.pdf)

该文件是从美国专利和商标局（USPTO）下载的，遵循与您的训练集相同的格式。

您之前使用Vision API文档文本检测过程从图像中提取了文本，并将文本包含在下面。

In [ ]:
test_item = "US010143036B2\n(12) United States Patent\nKhan\n(10) Patent No.: US 10,143,036 B2\n(45) Date of Patent: Nov. 27, 2018\n(54) MILLIMETER WAVE WIRELESS SYSTEM\nUSING LICENSED AND UNLICENSED\nFREQUENCY SPRECTRUM\n(58) Field of Classification Search\nCPC . H04W 74/006; H04W 80/02; H04W 34/005;\nHOW 84/04; HOW 84/045; H04W\n88/06; H04W 92/20; H04W 74/0816;\nHO4B 7/02\nSee application file for complete search history.\n(71) Applicant: Phazr, Inc., Allen, TX (US)\n(72) Inventor: Farooq Khan, Allen, TX (US)\n(56)\n(73) Assignee: Phazr, Inc., Allen, TX (US)\nReferences Cited\nU.S. PATENT DOCUMENTS\n(*) Notice:\nSubject to any disclaimer, the term of this\npatent is extended or adjusted under 35\nU.S.C. 154(b) by 44 days.\n10,057,898 B2 *\n10,075,852 B2 *\n2013/0072125 A1*\n8/2018 Khan\n9/2018 Nekovee\n3/2013 Yoon\nH04W 72/044\nH04W 16/28\nHO1P 1/10\n455/67.11\nH04B 1/40\n455/78\n(21) Appl. No.: 15/644,553\n2014/0106686 A1*\n4/2014 Higgins\n(22) Filed:\nJul. 7, 2017\n(Continued)\n(65)\nPrior Publication Data\nUS 2018/0035487 A1\nFeb. 1, 2018\nPrimary Examiner — Devan Sandiford\n(74) Attorney, Agent, or Firm — Michael A. Rahman\nRelated U.S. Application Data\n(60) Provisional application No. 62/369,038, filed on Jul.\n30, 2016.\n(51) Int. CI.\nH0W 84/04\n(2009.01)\nH04W 74/00\n(2009.01)\nH04W 84/00\n(2009.01)\nH04W 92/20\n(2009.01)\nH04W 80/02\n(2009.01)\nH0W 88/06\n(2009.01)\nH04B 7/02\n(2018.01)\nH04W 74/08\n(2009.01)\n(52) U.S. CI.\nCPC\nH04W 84/045 (2013.01); H04B 7/02\n(2013.01); H04W 74/006 (2013.01); H04W\n80/02 (2013.01); H04W 84/005 (2013.01);\nH04W 84/04 (2013.01); H04W 88/06\n(2013.01); H04W 92/20 (2013.01); H04W\n74/0816 (2013.01)\n(57)\nABSTRACT\nA method of wireless communication includes transmitting\nin a downlink direction on a licensed millimeter wave band,\nby a radio base station, a first millimeter wave band signal\nat high transmit equivalent isotropically radiated power\n(EIRP) using a multiple input multiple output transmit\nantenna array. The method includes receiving by a commu-\nnications device the first millimeter wave band signal. The\nmethod includes transmitting in an uplink direction on an\nunlicensed millimeter wave band, by the communications\ndevice, a second millimeter wave band signal at low transmit\nequivalent isotropically radiated power (EIRP) using a mul-\ntiple input multiple output transmit antenna array. The\nmethod includes receiving on the unlicensed millimeter\nwave band, by the radio base station, the second millimeter\nwave band signal at a high receive gain using a multiple\ninput multiple output receive antenna array.\n26 Claims, 11 Drawing Sheets\n100\nHigh-Gain\nantenna array\nHigh power\nPower amplifiers\nComplex\nTX\nTX\nLow Transmit\nEIRP\nLicensed\nSpectrum.\nRX\nLow Receive\nGain\nUnlicensed\nSpectrum\nHighly\nsensitive\nRX\nCommunication Device\nDevice\nC1\nLow-Noise\nFigure\nLNA\nAccess Point\nDevice\n00\nDevice\nC2\nAccess\nPoint\nAO\n"

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源上，您可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

     { 'content': text_string }

由于 predict() 方法可以接受多个项目（实例），请将您的单个测试项目作为一个包含一个测试项目的列表发送。

#### 响应

来自 predict() 调用的响应是一个 Python 字典，具有以下条目:

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个实体的类名。
- `confidences`: 每个实体的预测置信度，介于 0 和 1 之间。
- `textSegmentStartOffsets`: 实体在文本中的起始字符偏移量。
- `textSegmentEndOffsets`: 实体在文本中的结束字符偏移量。
- `deployed_model_id`: 执行预测的已部署 `Model` 资源的 Vertex AI 标识符。

In [ ]:
import json

instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(json.dumps(prediction, indent=4))

当你完成预测后，你可以将模型从`Endpoint`资源中取消部署。这将取消所有计算资源，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML or Pipeline training job
job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

print("Clean up completed!")